In [12]:
import pygsheets #Importing python in google sheets
import pandas as pd #importing pandas
from pandas import Series,DataFrame
import datetime as dt
from datetime import datetime, timedelta,date
import numpy as np


# ETM	Pilot Name	Mobile Number	alternate_number	

# Rating	Last Call	Todays Call	Yesterday Call Final OS


my_date = date.today() 

my_time = datetime.min.time()
my_datetime = datetime.combine(my_date, my_time)

week_start = my_date - timedelta(days=my_date.weekday())

week_end = week_start - timedelta(days=7)

week_end_datetime = datetime.combine(week_end, my_time)

today = my_datetime
yesterday = today - timedelta(days = 1)

tomorrow = today + timedelta(days = 1)

gc = pygsheets.authorize(service_file='client_secret.json')

car_status_report_sheet = gc.open("Car Status Report")

car_allotment_tab = car_status_report_sheet.worksheet_by_title("Mumbai")

car_allotment_data = pd.DataFrame(car_allotment_tab.get_all_records())


car_allotment_data = car_allotment_data[['Car Number','ETM']]

car_allotment_data['ETM']=car_allotment_data['ETM'].str.strip()

car_allotment_data['ETM']=car_allotment_data['ETM'].replace('-','unalloted')

car_allotment_data = car_allotment_data.groupby(['Car Number']).max()

car_allotment_data.reset_index(inplace = True)


fleet_driver_sheet = gc.open("Fleet_driver")

fleet_driver_tab = fleet_driver_sheet.worksheet_by_title("Fleet_driver")

fleet_driver = pd.DataFrame(fleet_driver_tab.get_all_records())

fleet_driver = fleet_driver[['employee_id','name','mobile','alternate_number']]


fleet_driver.rename(columns={"employee_id":"ETM"},inplace = True)


final_table = pd.merge(car_allotment_data,fleet_driver,on='ETM',how='left')

print(final_table.head())

commitment_mapping_master_sheet = gc.open("Commitment Mapping 3.0")

commitment_mapping_calling_tab = commitment_mapping_master_sheet.worksheet_by_title("Calling Data")

commitment_mapping_calling_data = pd.DataFrame(commitment_mapping_calling_tab.get_all_records())

commitment_mapping_calling_data = commitment_mapping_calling_data[['Timestamp','ETM ID','Final comment']]

commitment_mapping_calling_data.rename(columns={"ETM ID":"ETM"},inplace = True)

commitment_mapping_calling_data['Timestamp'] = pd.to_datetime(commitment_mapping_calling_data['Timestamp'],dayfirst = True)

max_call_data = commitment_mapping_calling_data[['Timestamp','ETM']]


max_call = max_call_data.groupby(['ETM']).max()


yesterday_commitment_calling_data = commitment_mapping_calling_data.copy()
today_commitment_calling_data = commitment_mapping_calling_data.copy()




today_commitment_calling_data['start date'] = today
today_commitment_calling_data['end date'] = tomorrow

print(today_commitment_calling_data.tail())



print(today_commitment_calling_data.tail())

today_commitment_calling_data = today_commitment_calling_data.loc[(today_commitment_calling_data['start date'] < today_commitment_calling_data['Timestamp']) & (today_commitment_calling_data['end date'] > today_commitment_calling_data['Timestamp'])]


print(today_commitment_calling_data.tail())




yesterday_commitment_calling_data['start date'] = yesterday
yesterday_commitment_calling_data['end date'] = today



yesterday_commitment_calling_data = yesterday_commitment_calling_data.loc[(yesterday_commitment_calling_data['start date'] < yesterday_commitment_calling_data['Timestamp']) & (yesterday_commitment_calling_data['end date'] > yesterday_commitment_calling_data['Timestamp'])]

print(yesterday_commitment_calling_data.head())

today_commitment_calling_data = today_commitment_calling_data[['ETM','Final comment']]

today_commitment_calling_data.rename(columns={"Final comment":"Today Call"},inplace = True)


today_call = today_commitment_calling_data.groupby(['ETM']).max()

yesterday_commitment_calling_data = yesterday_commitment_calling_data[['ETM','Final comment']]

yesterday_commitment_calling_data.rename(columns={"Final comment":"Yesterday Call"},inplace = True)

yesterday_call = yesterday_commitment_calling_data.groupby(['ETM']).max()


final_table = pd.merge(final_table,max_call,on='ETM',how='left')
final_table = pd.merge(final_table,today_call,on='ETM',how='left')
final_table = pd.merge(final_table,yesterday_call,on='ETM',how='left')

print(final_table.head())




cms_sheet = gc.open("Car Master Sheet")

hisaab_tab = cms_sheet.worksheet_by_title("Driver Hisaab Final")

hisaab_data = pd.DataFrame(hisaab_tab.get_all_records())

hisaab_data = hisaab_data[['Driver ETM','Final OS']]



hisaab_data.rename(columns={"Driver ETM":"ETM"},inplace = True)



final_table = pd.merge(final_table,hisaab_data,on='ETM',how='left')


asr_mumbai_sheet = gc.open("Allotment Status Report")

asr_mumbai = asr_mumbai_sheet.worksheet_by_title("Allotment Status Report")

allotment_mumbai = pd.DataFrame(asr_mumbai.get_all_records())

allotment_mumbai = allotment_mumbai[['Active or not','Timestamp','ID']]

allotment_mumbai['Timestamp'] = pd.to_datetime(allotment_mumbai['Timestamp'],dayfirst = True)

allotment_mumbai.rename(columns={"ID":"ETM"},inplace = True)


allotment_mumbai = allotment_mumbai.loc[allotment_mumbai['Active or not'] == 'Active']
allotment_mumbai = allotment_mumbai[['Timestamp','ETM']]
first_allotment = allotment_mumbai.groupby(['ETM']).min()
print(first_allotment.head())
first_allotment['end_date'] = week_end_datetime
first_allotment = first_allotment.loc[(first_allotment['end_date'] < first_allotment['Timestamp'])]
first_allotment['Rating'] = 'New'
first_allotment.drop(['Timestamp','end_date'],axis = 1, inplace = True)
print(first_allotment.head())
last_allotment = allotment_mumbai.groupby('ETM').max()
last_allotment.rename(columns={"Timestamp":"Last Allotment Date"},inplace = True)
print(last_allotment.head())
final_table = pd.merge(final_table,first_allotment,on='ETM',how='left')
final_table = pd.merge(final_table,last_allotment,on='ETM',how='left')
print(final_table.head())
final_table["Final OS"].fillna(0,inplace = True)
final_table.fillna("",inplace = True)
final_table.rename(columns={"Car Number":"car_number","Timestamp":"Last Call Time"},inplace = True)


#Commitment Mapping 3.0
commitment_mapping_master_sheet = gc.open("Commitment Mapping 3.0")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Master")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number','Last Week Trips']]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

#rating piker laggard calculation.

commitmentmapping_table['Last Week Trips']=commitmentmapping_table['Last Week Trips'].replace(np.nan,0).replace('',0)
commitmentmapping_table['Last Week Trips'] = commitmentmapping_table['Last Week Trips'].astype(float)
aa= []
for i in commitmentmapping_table.index:
    if commitmentmapping_table['Rating'].values[i]=='New':
        aa.append('New')
    elif commitmentmapping_table['Last Week Trips'].values[i] == 0:
        aa.append('ND')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 30.0:
        aa.append('Piker')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 55.0:
        aa.append('Laggard')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 75.0:
        aa.append('Mediocre') 
    elif commitmentmapping_table['Last Week Trips'].values[i] < 95.0:
        aa.append('Performer')
    elif commitmentmapping_table['Last Week Trips'].values[i] >= 95.0:
        aa.append('out Performer')
    else:
        aa.append('')
commitmentmapping_table['Rating']=aa[0:commitmentmapping_table.shape[0]]
commitmentmapping_table


commitmentmapping_table['car_number'] = commitmentmapping_table['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitmentmapping_table = commitmentmapping_table[commitmentmapping_table['car_number'].notna()]

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#roarings
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Roaring Lions")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Roaring_Lions")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#terrific
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Terrific Tigers")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Terrific_Tigers")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#black
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Black Panthers")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Black_Panthers")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#silent
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Silent Killers")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Silent_Killers")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#deep
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Deep Hunters")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Deep_Hunters")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
final_OS_table = commitmentmapping_table[['Final OS']]
last_allotment = commitmentmapping_table[['Last Allotment Date']]
commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

   Car Number       ETM                       name        mobile  \
0  MH01CR9416    Repair                        NaN           NaN   
1  MH01CR9417  ETM16692  Balaji Nagorao Chapkanade  7.304866e+09   
2  MH01CR9418  ETM01386              Vijay Kumar -  9.867611e+09   
3  MH01CR9419  ETM10739              Sunil Goswami  6.367465e+09   
4  MH01CR9421    Repair                        NaN           NaN   

  alternate_number  
0              NaN  
1       8862038044  
2       7054177758  
3       8426850360  
4              NaN  
                Timestamp       ETM        Final comment start date   end date
60153 2022-08-24 19:23:30  ETM24596        Not Connected 2022-08-24 2022-08-25
60154 2022-08-24 19:27:41  ETM18207         Health Issue 2022-08-24 2022-08-25
60155 2022-08-24 19:37:55  ETM13375  Need to recover car 2022-08-24 2022-08-25
60156 2022-08-24 19:49:20  ETM23803               Online 2022-08-24 2022-08-25
60157 2022-08-24 19:51:04  ETM03458        Not Connected 2022-08-24 20

In [2]:
# pd.set_option('display.max_rows', None)

In [65]:
#Commitment Mapping 3.0
commitment_mapping_master_sheet = gc.open("Commitment Mapping 3.0")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Master")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number','Last Week Trips']]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')

#rating piker laggard calculation.

commitmentmapping_table['Last Week Trips']=commitmentmapping_table['Last Week Trips'].replace(np.nan,0).replace('',0)
commitmentmapping_table['Last Week Trips'] = commitmentmapping_table['Last Week Trips'].astype(float)
aa= []
for i in commitmentmapping_table.index:
    if commitmentmapping_table['Rating'].values[i]=='New':
        aa.append('New')
    elif commitmentmapping_table['Last Week Trips'].values[i] == 0:
        aa.append('ND')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 30.0:
        aa.append('Piker')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 55.0:
        aa.append('Laggard')
    elif commitmentmapping_table['Last Week Trips'].values[i] < 75.0:
        aa.append('Mediocre') 
    elif commitmentmapping_table['Last Week Trips'].values[i] < 95.0:
        aa.append('Performer')
    elif commitmentmapping_table['Last Week Trips'].values[i] >= 95.0:
        aa.append('out Performer')
    else:
        aa.append('')
commitmentmapping_table['Rating']=aa[0:commitmentmapping_table.shape[0]]
commitmentmapping_table


commitmentmapping_table['car_number'] = commitmentmapping_table['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitmentmapping_table = commitmentmapping_table[commitmentmapping_table['car_number'].notna()]

# ETM_table = commitmentmapping_table[['ETM','name','mobile','alternate_number','Rating','Last Call Time','Today Call','Yesterday Call']]
# final_OS_table = commitmentmapping_table[['Final OS']]
# last_allotment = commitmentmapping_table[['Last Allotment Date']]
# commitment_mapping_master_tab.set_dataframe(ETM_table,'E1')
# commitment_mapping_master_tab.set_dataframe(final_OS_table,'V1')
# commitment_mapping_master_tab.set_dataframe(last_allotment,'Z1')

#roarings
commitment_mapping_master_sheet = gc.open("Commitment Mapping - Roaring Lions")
commitment_mapping_master_tab = commitment_mapping_master_sheet.worksheet_by_title("Roaring_Lions")
commitment_mapping_master = pd.DataFrame(commitment_mapping_master_tab.get_all_records())
commitment_mapping_master = commitment_mapping_master[['car_number']]
commitment_mapping_master['car_number'] = commitment_mapping_master['car_number'].replace(r'^\s*$', np.nan, regex=True)
commitment_mapping_master = commitment_mapping_master[commitment_mapping_master['car_number'].notna()]
commitmentmapping_table = pd.merge(commitment_mapping_master,final_table,on='car_number',how='left')
commitmentmapping_table

,car_number,Last Week Trips,ETM,name,mobile,alternate_number,Last Call Time,Today Call,Yesterday Call,Final OS,Rating,Last Allotment Date
0,MH01CV4503,0.0,ETM20403,Somnath Devram Gaikwad,9082648741.0,8308233460,NaT,,,0.0,ND,NaT
1,MH01CV5215,0.0,Insurance,,,,2022-12-04 16:29:37,,,0.0,ND,2022-07-09 15:36:42
2,MH01CV5222,15.0,Repairs,,,,NaT,,,0.0,Piker,NaT
3,MH01CV5222,15.0,Repairs,,,,NaT,,,0.0,Piker,NaT
4,MH01CV7753,46.0,ETM11498,Riyaz Ahmed Shaikh,9768931005.0,7972619479,NaT,,,0.0,Laggard,NaT
5,MH01CV7759,66.0,ETM05062,Avil Kamble,9969237889.0,9594530605,NaT,,,-2919.0,New,2022-07-08 14:12:50
6,MH01CV5214,16.0,Repairs,,,,NaT,,,0.0,Piker,NaT
7,MH01CV5214,16.0,Repairs,,,,NaT,,,0.0,Piker,NaT
8,MH01CV8568,4.0,Parking,,,,2022-12-04 09:59:04,,,0.0,Piker,NaT
9,MH01CV9667,0.0,ETM23190,Namdeo Mahadev Netke,9372307908.0,9152900549,NaT,,,0.0,New,2022-07-13 20:25:22


In [63]:
commitmentmapping_table.head(3)

,car_number,Last Week Trips,ETM,name,mobile,alternate_number,Last Call Time,Today Call,Yesterday Call,Final OS,Rating,Last Allotment Date
0,MH01CV4503,0.0,ETM20403,Somnath Devram Gaikwad,9082648741.0,8308233460,NaT,,,0.0,,NaT
1,MH01CV5215,0.0,Insurance,,,,2022-12-04 16:29:37,,,0.0,,2022-07-09 15:36:42
2,MH01CV5222,15.0,Repairs,,,,NaT,,,0.0,,NaT


In [2]:
#importing libraries

import pandas as pd
import numpy as np
import pandasql as ps
import datetime
from datetime import date, timedelta
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pygsheets
import warnings
warnings.filterwarnings("ignore")
from twilio.rest import Client as c


#creating connections

scope= ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials= ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scope) 
client= gspread.authorize(credentials)

clients = pygsheets.authorize(service_file='client_secret.json')

#sheet keys

car_status_report='1CfqvArNmTofvNOFAhQ965B7cMA7lX40e3RZZKY6IUjI'
fleet_driver='1qceRS8LU17n5YWvgewcpJZif02KxV8Mtvs6anomHD80'
allotment_status_report='1cpR6AVVpk9TF4_I38IFYPPOqk-_bSROHgVVYdaXLXOI'
car_master='1_r5OMN1P8Tof5IRaE5jYd-jHaP8j15avQVMycDJdhec'
car_servicing_schedule_calling_servicing_tab='11WVBiisNIF8Xb7sx7GZEok7mG4Lz3uo96-e00xVtW0M'
commitment_mapping_3='1sy3Gxrnh8bX6ibpGR6X8rba8H2IfeSNdBGT7CIFi3SY'

#5 sheet keys

terrific='1f-DJ5O3zKKkAtXfXQghWzbsoXyy-ipdJ5q7Yma4tF9M'
roaring='1MiZZY9MPEhhg-B9LyfmThQGV3Dy3XnKpb5NqgaYFwIQ'
silent='1ZwXyZPyt7qhjTO5VkfwoGTJ48fmg_38MTdy4EHdVhwA'
deep='1TPRKPsQfy4qY19byAuYi0Ie3Xld7ddrrBS8xqhggtcM'
black='1KD0ABDWp3YvqIkmoQAZKRS-7-S8i-HtlO4KKpBMB1tU'

#########################################################################sheets
try:
    
    #car_status_report

    sheet= client.open_by_key(car_status_report)
    ws= sheet.worksheet('Cars')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
    dfs['Current DM']=dfs['Current DM'].str.title()
    reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
    reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev']
    car_status_df=reject_df_1.iloc[:,[1,0,4]]
    car_status_df['ETM']=car_status_df['ETM'].str.strip()
    car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
    car_status_df

    #etm from master
    
    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Master')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    masterview_car_no=dfm.iloc[:,[2]]
    masterview_car_no
        
    #fleet_driver
    
    sheet= client.open_by_key(fleet_driver)
    ws= sheet.worksheet('Fleet_driver')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    fleet_driver_dfs=dfm.iloc[:,[2,3,5,6]]
    fleet_driver_df=fleet_driver_dfs.loc[fleet_driver_dfs['employee_id'] != '']
    fleet_driver_df['employee_id']=fleet_driver_df['employee_id'].str.strip()
    fleet_driver_df.rename({'employee_id':'ETM','name':'Pilot Name','mobile':'Mobile Number'},axis=1,inplace=True)
    fleet_driver_df

    #allotment_status_report

    sheet= client.open_by_key(allotment_status_report)
    ws= sheet.worksheet('Allotment Status Report')
    data = ws.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data,columns=headers)
    allt_df=df.loc[:,['Timestamp','ID']]
    allt_df['ID']=allt_df['ID'].str.upper()
    allot_df=allt_df[allt_df.ID.str.startswith('ETM')]
    today = datetime.date.today()
    previous_monday = today - datetime.timedelta(days=today.weekday())
    previous_monday_week= previous_monday - datetime.timedelta(days=7)
    allot_df['Timestamp']=pd.to_datetime(allot_df['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')
    df_thisweek = allot_df[(allot_df['Timestamp'] >= pd.to_datetime(previous_monday_week)) & (allot_df['Timestamp'] <= pd.to_datetime(today))]
    df_1_week=df_thisweek.groupby(['ID'],as_index=True)['Timestamp'].min()
    allotment_s_r=pd.DataFrame(df_1_week).reset_index()
    allotment_s_r.rename({'ID':'ETM','Timestamp':'Rating'},axis=1,inplace=True)
    allotment_s_r['Rating']='New'
    allotment_s_r

    #commitment_mapping_calling_last call

    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Calling Data')
    data = ws.get_all_values()
    headers = data.pop(0)
    commitment_mapping_calling_tab = pd.DataFrame(data,columns=headers)
    commitment_mapping=commitment_mapping_calling_tab.loc[:,['Timestamp','ETM ID']]
    commitment_mapping['Timestamp']=commitment_mapping['Timestamp']
    commitment_mapping['Timestamp'] = pd.to_datetime(commitment_mapping['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')#27/12/2021
    commitment_mapping_calling_tab_last_call=commitment_mapping.groupby(['ETM ID'],as_index=True)['Timestamp'].max()
    commitment_mapping_calling_tab_last_call_df=pd.DataFrame(commitment_mapping_calling_tab_last_call).reset_index()
    commitment_mapping_calling_tab_last_call_df.rename({'ETM ID':'ETM','Timestamp':'Last Call'},axis=1,inplace=True)
    commitment_mapping_calling_tab_last_call_df

    #commitment_mapping_todays_call

    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Calling Data')
    data = ws.get_all_values()
    headers = data.pop(0)
    commitment_mapping_calling_tab = pd.DataFrame(data,columns=headers)
    commitment_mapping=commitment_mapping_calling_tab.iloc[:,[0,3,7]]
    today = datetime.date.today()
    commitment_mapping['Timestamp']=pd.to_datetime(commitment_mapping['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')
    commitment_mapping['Timestamp'] = commitment_mapping['Timestamp'].dt.strftime('%Y-%m-%d')
    df_thisweek = commitment_mapping[(commitment_mapping['Timestamp'] == today.strftime('%Y-%m-%d'))]
    commitment_mapping_calling_tab_today_call=df_thisweek.loc[:,['ETM ID','Driver Status?']]
    commitment_mapping_calling_tab_today_call.rename({'ETM ID':'ETM','Driver Status?':'Todays Call'},axis=1,inplace=True)
    commitment_mapping_calling_tab_today_call

    #commitment_mapping_calling_yesterday_call

    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Calling Data')
    data = ws.get_all_values()
    headers = data.pop(0)
    commitment_mapping_calling_tab = pd.DataFrame(data,columns=headers)
    commitment_mapping=commitment_mapping_calling_tab.iloc[:,[0,3,7]]
    today = datetime.date.today()
    yesterday = today- timedelta(days=1)
    commitment_mapping['Timestamp']=pd.to_datetime(commitment_mapping['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')
    commitment_mapping['Timestamp'] = commitment_mapping['Timestamp'].dt.strftime('%Y-%m-%d')
    df_thisweek = commitment_mapping[(commitment_mapping['Timestamp'] == yesterday.strftime('%Y-%m-%d'))]
    commitment_mapping_calling_tab_yesterday_call=df_thisweek.loc[:,['ETM ID','Driver Status?']]
    commitment_mapping_calling_tab_yesterday_call.rename({'ETM ID':'ETM','Driver Status?':'Yesterday Call'},axis=1,inplace=True)
    commitment_mapping_calling_tab_yesterday_call

    #car_master

    sheet= client.open_by_key(car_master)
    ws= sheet.worksheet('Driver Hisaab Final')
    data = ws.get_all_values()
    headers = data.pop(0)
    car_master_final= pd.DataFrame(data,columns=headers)
    car_master_final_df=car_master_final.iloc[:,[0,1]]
    car_master_final_df.rename({'Driver ETM':'ETM'},axis=1,inplace=True)
    car_master_final_df

    #allotment status report for allotment date

    sheet= client.open_by_key(allotment_status_report)
    ws= sheet.worksheet('Allotment Status Report')
    data = ws.get_all_values()
    headers = data.pop(0)
    df = pd.DataFrame(data,columns=headers)
    allt_df=df.loc[:,['Timestamp','Car Number','ID']]
    allt_df['ID']=allt_df['ID'].str.upper()
    allot_df=allt_df[allt_df.ID.str.startswith('ETM')]
    allot_max=allot_df.groupby(['Car Number','ID'],as_index=True)['Timestamp'].max()
    allot_max_dfm=pd.DataFrame(allot_max).reset_index()
    allot_max_dfm.rename({'ID':'ETM','Timestamp':'Allotment date'},axis=1,inplace=True)
    allot_max_df= allot_max_dfm[['Allotment date', 'ETM']].groupby(['ETM'])['Allotment date'].max()
    allot_max_df
    
    
###############################################################################pushing new etm mapping with car number

#     #car_status_report

#     sheet= client.open_by_key(car_status_report)
#     ws= sheet.worksheet('Cars')
#     data = ws.get_all_values()
#     headers = data.pop(0)
#     dfm = pd.DataFrame(data,columns=headers)
#     dfs=dfm.loc[dfm['Type'].isin(['Revenue Share'])]
#     dfs['Current DM']=dfs['Current DM'].str.title()
#     reject_df=dfs.loc[dfs['Current DM'] != 'Pune']
#     reject_df_1=reject_df.loc[reject_df['Current DM'] != 'Ev']
#     car_status_df=reject_df_1.iloc[:,[1,0,4]]
#     car_status_df['ETM']=car_status_df['ETM'].str.strip()
#     car_status_df.rename({'Current DM':'Team Name','Car Number':'car_number'},axis=1,inplace=True)
#     car_status_df

#     #etm from master
    
#     sheet= client.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet('Master')
#     data = ws.get_all_values()
#     headers = data.pop(0)
#     dfm = pd.DataFrame(data,columns=headers)
#     masterview_car_no=dfm.iloc[:,[2]]
#     masterview_car_no
    
#     etm_dfs=masterview_car_no.merge(car_status_df,on='car_number',how='left')
#     new_etm=etm_dfs.iloc[:,[2]]  
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.clear(start='E',end='E')
#     ws.set_dataframe(new_etm,(1,5)) 
    
#     ###################################################################### master etm column
    
#     #etm from master
#     sheet= client.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet('Master')
#     data = ws.get_all_values()
#     headers = data.pop(0)
#     dfm = pd.DataFrame(data,columns=headers)
#     etm_df=dfm.iloc[:,[1,4]]
#     etm_df['ETM']=etm_df['ETM'].str.strip()
#     etm_df

    #last week trip from master
    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Master')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    last_week=dfm.iloc[:,[1,4,8,13]]
    last_week
    
# #     ##################################################################### mapping on the basis of etm and 5 sheets

#     #fleet_driver 

#     cs_flt=etm_df.merge(fleet_driver_df,on='ETM',how='left')
#     cs_flt_d=cs_flt.iloc[:,[2,3,4]]
#     cs_flt_df=cs_flt_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_flt_df,(1,6))
    
#     #rating

#     cs_allot_rating=etm_df.merge(allotment_s_r,on='ETM',how='left')
#     cs_allot_rating_d=cs_allot_rating.iloc[:,[2]]
#     cs_allot_rating_df=cs_allot_rating_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_allot_rating_df,(1,9))    
    
    #rating calculation

    last_week['Last Week Trips']=last_week['Last Week Trips'].replace(np.nan,0).replace('',0)
    last_week['Last Week Trips'] = last_week['Last Week Trips'].astype(float)
    aa= []
    for i in last_week.index:
        if last_week['Rating'].values[i]=='New':
            aa.append('New')
        elif last_week['Last Week Trips'].values[i] == 0:
            aa.append('ND')
        elif last_week['Last Week Trips'].values[i] < 30.0:
            aa.append('Piker')
        elif last_week['Last Week Trips'].values[i] < 55.0:
            aa.append('Laggard')
        elif last_week['Last Week Trips'].values[i] < 75.0:
            aa.append('Mediocre') 
        elif last_week['Last Week Trips'].values[i] < 95.0:
            aa.append('Performer')
        elif last_week['Last Week Trips'].values[i] >= 95.0:
            aa.append('out Performer')
        else:
            aa.append('')
    last_week['Rating']=aa[0:last_week.shape[0]]
    last_week_df=last_week.iloc[:,[1]]
    
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(last_week_df,(1,9))    
    
#     #commitment_mapping_calling_last call

#     cs_clc=etm_df.merge(commitment_mapping_calling_tab_last_call_df,on='ETM',how='left')
#     cs_clc_d=cs_clc.iloc[:,[2]]
#     cs_clc_df=cs_clc_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_clc_df,(1,10))
    
#     #commitment_mapping_calling_tab_today_call

#     cs_ctc=etm_df.merge(commitment_mapping_calling_tab_today_call,on='ETM',how='left')
#     cs_ctc_d=cs_ctc.iloc[:,[2]]
#     cs_ctc_df=cs_ctc_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_ctc_df,(1,11))
    
#     #commitment_mapping_calling_tab_yesterday_call

#     cs_cyc=etm_df.merge(commitment_mapping_calling_tab_yesterday_call,on='ETM',how='left')
#     cs_cyc_d=cs_cyc.iloc[:,[2]]
#     cs_cyc_df=cs_cyc_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_cyc_df,(1,12))
    
#     #car_master_final_df

#     cs_final=etm_df.merge(car_master_final_df,on='ETM',how='left')
#     cs_final_d=cs_final.iloc[:,[2]]
#     cs_final_df=cs_final_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_final_df,(1,22))
    
#     #allotment date

#     cs_allot=etm_df.merge(allot_max_df,on='ETM',how='left')
#     cs_allot_d=cs_allot.iloc[:,[2]]
#     cs_allot_df=cs_allot_d.fillna('')
#     sheet= clients.open_by_key(commitment_mapping_3)
#     ws= sheet.worksheet_by_title('Master')
#     ws.set_dataframe(cs_allot_df,(1,26))
            
        
    print("Commitment mapping etm updated succesfully")
    
except Exception as e:
    number=['+91 81084 16708','+91 98200 66683']
    for to_number in number:
        account_sid="AC3459ee86068c97f9cd2de30ad98146e4"
        auth_token="208bac357247abfae8b25d89406dc608"
        client=c(account_sid,auth_token)
        from_number='+1 9705577507'
        error=str(e)
        error_message=error+" error in your commitment mapping etm code"        
        client.messages.create(
            body=error_message,
            from_=from_number,
            to=to_number)
        print(e,"error in your commitment mapping etm code")

Commitment mapping etm updated succesfully


In [36]:
    #last week trip from master
    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Master')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    last_week=dfm.iloc[:,[1,4,8,13]]
    last_week

,Team Name,ETM,Rating,Last Week Trips
0,Deep Hunters,ETM19041,,12
1,Silent Killers,ETM20403,,0
2,Terrific Tigers,ETM10739,,18
3,Black Panthers,ETM10842,,0
4,Roaring Lions,ETM14533,,47
5,Roaring Lions,ETM14533,,10
6,Silent Killers,ETM22651,New,0
7,Silent Killers,Insurance,,15
8,Roaring Lions,ETM19606,,17
9,Terrific Tigers,Parking,,0


In [53]:
car_s_r=car_status_df.merge(fleet_driver_df,on='ETM',how='left')

df=car_s_r.loc[:,['car_number','ETM','Pilot Name','Mobile Number','alternate_number']]

df1=df.merge(commitment_mapping_calling_tab_last_call_df,on='ETM',how='left')

df2=df1.merge(commitment_mapping_calling_tab_today_call,on='ETM',how='left')

df3=df2.merge(commitment_mapping_calling_tab_yesterday_call,on='ETM',how='left')

df4=df3.merge(car_master_final_df,on='ETM',how='left')

df5=df4.merge(allot_max_df,on='ETM',how='left')

df6=df5.merge(last_week_df,on='ETM',how='left')

df6.fillna('',inplace=True)

# df6['Last Call']=df6['Last Call'].replace('NaT','')

df6['Last Call']=df6['Last Call'].astype(str)

df6['Last Call'].replace('NaT','', inplace=True,regex=True)

df6

sheet= clients.open_by_key(car_status_report)
ws= sheet.worksheet_by_title('etm')
ws.set_dataframe(df6,(1,1))

In [41]:
    #last week trip from master
    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Master')
    data = ws.get_all_values()
    headers = data.pop(0)
    dfm = pd.DataFrame(data,columns=headers)
    last_week=dfm.iloc[:,[1,4,8,13]]
    last_week

    last_week['Last Week Trips']=last_week['Last Week Trips'].replace(np.nan,0).replace('',0)
    last_week['Last Week Trips'] = last_week['Last Week Trips'].astype(float)
    aa= []
    for i in last_week.index:
        if last_week['Rating'].values[i]=='New':
            aa.append('New')
        elif last_week['Last Week Trips'].values[i] == 0:
            aa.append('ND')
        elif last_week['Last Week Trips'].values[i] < 30.0:
            aa.append('Piker')
        elif last_week['Last Week Trips'].values[i] < 55.0:
            aa.append('Laggard')
        elif last_week['Last Week Trips'].values[i] < 75.0:
            aa.append('Mediocre') 
        elif last_week['Last Week Trips'].values[i] < 95.0:
            aa.append('Performer')
        elif last_week['Last Week Trips'].values[i] >= 95.0:
            aa.append('out Performer')
        else:
            aa.append('')
    last_week['Rating']=aa[0:last_week.shape[0]]
    last_week_df=last_week.iloc[:,[1,2]]
    last_week_df

,ETM,Rating
0,ETM19041,Piker
1,ETM20403,ND
2,ETM10739,Piker
3,Repairs,ND
4,Repairs,Laggard
5,ETM14533,Piker
6,ETM22651,New
7,Insurance,Piker
8,Repairs,Piker
9,Parking,ND


In [21]:
    #commitment_mapping_todays_call

    sheet= client.open_by_key(commitment_mapping_3)
    ws= sheet.worksheet('Calling Data')
    data = ws.get_all_values()
    headers = data.pop(0)
    commitment_mapping_calling_tab = pd.DataFrame(data,columns=headers)
    commitment_mapping=commitment_mapping_calling_tab.iloc[:,[0,3,7]]
    today = datetime.date.today()
    commitment_mapping['Timestamp']=pd.to_datetime(commitment_mapping['Timestamp'], errors='coerce',format='%d/%m/%Y %H:%M:%S')
    commitment_mapping['Timestamp'] = commitment_mapping['Timestamp'].dt.strftime('%Y-%m-%d')
    df_thisweek = commitment_mapping[(commitment_mapping['Timestamp'] == today.strftime('%Y-%m-%d'))]
    commitment_mapping_calling_tab_today_call=df_thisweek.loc[:,['ETM ID','Driver Status?']]
    commitment_mapping_calling_tab_today_call.rename({'ETM ID':'ETM','Driver Status?':'Todays Call'},axis=1,inplace=True)
    commitment_mapping_calling_tab_today_call
    
    sheet= clients.open_by_key(car_status_report)
    ws= sheet.worksheet_by_title('test')
    ws.set_dataframe(commitment_mapping_calling_tab_today_call,(1,1))

In [8]:
pd.set_option('display.max_rows', None)